# Libraries

In [2]:
import pickle
import pandas as pd
import spacy
import spacymoji
from tqdm.notebook import tqdm
import numpy as np
import warnings
import os

from gensim.corpora import Dictionary
from gensim.models.wrappers import LdaMallet
import gensim

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# import pyLDAvis
# import pyLDAvis.gensim
# pyLDAvis.enable_notebook()

# Import

In [3]:
all_data = pd.read_csv("Data/all_tweets_sorted_fixed_date.csv", index_col=[0])
print("Original data combined with scraped data")
all_data.iloc[range(20, 25)]

Original data combined with scraped data


,date,id,tweet,sentiment
20,2020-03-20,1.240739e+18,Well one positive thing about this Corona Viru...,0.000000
21,2020-03-20,1.240739e+18,Due to the growing health concerns of the Coro...,0.279762
22,2020-03-20,1.240739e+18,Hopefully Soon Ass This Corona Virus 🦠 Bullshi...,0.000000
23,2020-03-20,1.240739e+18,Friends don’t let friends get fat during Coron...,0.000000
24,2020-03-20,1.240740e+18,#Repost cristi_christensen\n・・・\nTake that Cor...,0.187500


In [4]:
tweets = all_data["tweet"]

In [5]:
nlp = spacy.load("en_core_web_sm")
docs = [post for post in tqdm(nlp.pipe(tweets, disable=["ner", "parser"]))]
lemmas = [[t.lemma_ for t in doc if not t.is_punct and not t.is_stop and t.pos_ in ["NOUN", "VERB", "ADJ"]] for doc in tqdm(docs)]

In [6]:
lemmas_nouns = [[t.lemma_ for t in doc if not t.is_punct and not t.is_stop and t.pos_ in ["NOUN"]] for doc in tqdm(docs)]

# LDA

Skip code below till loop

In [ ]:
# MIN_DF = 20 # minium document frequency
# MAX_DF = 0.6 # maximum document frequency

# dictionary = Dictionary(lemmas) # get the vocabulary
# dictionary.filter_extremes(no_below=MIN_DF, 
#                            no_above=MAX_DF)
# corpus = [dictionary.doc2bow(text) for text in lemmas]

# PATH_TO_MALLET = "mallet-2.0.8/bin/mallet"
# N_TOPICS = 20
# N_ITERATIONS = 10000

# # TAKES LONG!
# lda = LdaMallet(PATH_TO_MALLET,
#                 corpus=corpus,
#                 id2word=dictionary,
#                 num_topics=N_TOPICS,
#                 iterations=N_ITERATIONS)

# lda.save(f"Models/lda{N_TOPICS}_{N_ITERATIONS}i")

In [ ]:
# dictionary = Dictionary(lemmas_nouns) # get the vocabulary
# dictionary.filter_extremes(no_below=MIN_DF, 
#                            no_above=MAX_DF)
# corpus = [dictionary.doc2bow(text) for text in lemmas_nouns]

# PATH_TO_MALLET = "mallet-2.0.8/bin/mallet"
# N_TOPICS = 20
# N_ITERATIONS = 10000

# # TAKES LONG!
# lda_nouns = LdaMallet(PATH_TO_MALLET,
#                 corpus=corpus,
#                 id2word=dictionary,
#                 num_topics=N_TOPICS,
#                 iterations=N_ITERATIONS)

# lda_nouns.save(f"Models/lda{N_TOPICS}_{N_ITERATIONS}i_nouns")

In [ ]:
# lda = LdaMallet.load(f"Models/lda{N_TOPICS}_{N_ITERATIONS}i_nouns")

### LOOP

In [24]:
N_TOPICS = [35, 50]
NOUNS_ONLY = [False]
N_ITERATIONS = [10000]
MIN_DF = 20 # minium document frequency
MAX_DF = 0.6 # maximum document frequency  
PATH_TO_MALLET = "mallet-2.0.8/bin/mallet"

In [25]:
for n_topics in N_TOPICS:
    for n_iterations in N_ITERATIONS:
        for nouns_only in NOUNS_ONLY:
            if nouns_only == False:
                nouns_only = ""
                lemmas_ = lemmas
            if nouns_only == True:
                nouns_only = "_nouns"
                lemmas_ = lemmas_nouns
            save_name = f"Models/lda_{n_topics}t_{n_iterations}i"+nouns_only
            if not os.path.isfile(save_name):
                dictionary = Dictionary(lemmas_) # get the vocabulary
                dictionary.filter_extremes(no_below=MIN_DF, 
                                           no_above=MAX_DF)
                corpus = [dictionary.doc2bow(text) for text in lemmas_]

                # TAKES LONG!
                lda = LdaMallet(PATH_TO_MALLET,
                                corpus=corpus,
                                id2word=dictionary,
                                num_topics=n_topics,
                                iterations=n_iterations)

                lda.save(save_name)
            print("DONE: "+save_name)
print("COMPLETELY DONE")

DONE: Models/lda_35t_10000i
DONE: Models/lda_50t_10000i
COMPLETELY DONE


# Top 10 words

In [26]:
# for n_topics in N_TOPICS:
#     for n_iterations in N_ITERATIONS:
#         for nouns_only in NOUNS_ONLY:
#             if nouns_only == False:
#                 nouns_only = ""
#             if nouns_only == True:
#                 nouns_only = "_nouns"
#             save_name = f"Models/lda_{n_topics}t_{n_iterations}i"+nouns_only
#             print("\n"+save_name)
#             lda = LdaMallet.load(save_name)
#             top_10 = []
#             for topic in range(n_topics):
#                 words = lda.show_topic(topic, 10)
#                 topic_n_words = ', '.join([word[0] for word in words])
#                 temp_top_10 = 'Topic {}: {}'.format(str(topic), topic_n_words)
#                 top_10.append(temp_top_10)
#                 print(temp_top_10)

In [27]:
# top_10 = []
# for topic in range(N_TOPICS):
#     words = lda.show_topic(topic, 10)
#     topic_n_words = ', '.join([word[0] for word in words])
#     temp_top_10 = 'Topic {}: {}'.format(str(topic), topic_n_words)
#     top_10.append(temp_top_10)
#     print(temp_top_10)

In [28]:
# top_10 = []
# for topic in range(N_TOPICS):
#     words = lda_nouns.show_topic(topic, 10)
#     topic_n_words = ', '.join([word[0] for word in words])
#     temp_top_10 = 'Topic {}: {}'.format(str(topic), topic_n_words)
#     top_10.append(temp_top_10)
#     print(temp_top_10)

In [29]:
# all_data.head()

In [30]:
# topic_distributions.head()

# Combine all_data with topics per tweet

In [31]:
# transformed_docs = lda.load_document_topics()
# topic_distributions = pd.DataFrame([[x[1] for x in doc] for doc in transformed_docs], 
#                                    columns=[i for i in range(N_TOPICS)])
# topic_columns = ["topic_"+str(col) for col in topic_distributions.columns]
# topic_distributions.columns = topic_columns
# all_data_topic = all_data.join(topic_distributions).reset_index(drop=True)
# all_data_topic.to_csv("Data/tweet_topic.csv", index=False)
# all_data_topic.iloc[range(10, 15)]

In [32]:
# transformed_docs = lda_nouns.load_document_topics()
# topic_distributions = pd.DataFrame([[x[1] for x in doc] for doc in transformed_docs], 
#                                    columns=[i for i in range(N_TOPICS)])
# topic_columns = ["topic_"+str(col) for col in topic_distributions.columns]
# topic_distributions.columns = topic_columns
# all_data_topic = all_data.join(topic_distributions).reset_index(drop=True)
# all_data_topic.to_csv("Data/tweet_nouns_topic.csv", index=False)
# all_data_topic.iloc[range(10, 15)]

In [33]:
for n_topics in N_TOPICS:
    for n_iterations in N_ITERATIONS:
        for nouns_only in NOUNS_ONLY:
            if nouns_only == False:
                nouns_only = ""
            if nouns_only == True:
                nouns_only = "_nouns"
            save_name = f"Models/lda_{n_topics}t_{n_iterations}i"+nouns_only
            print("\n"+save_name)
            lda = LdaMallet.load(save_name)
            
            transformed_docs = lda.load_document_topics()
            topic_distributions = pd.DataFrame([[x[1] for x in doc] for doc in transformed_docs], 
                                               columns=[i for i in range(n_topics)])
            topic_columns = ["topic_"+str(col) for col in topic_distributions.columns]
            topic_distributions.columns = topic_columns
            all_data_topic = all_data.join(topic_distributions).reset_index(drop=True)
            all_data_topic.to_csv(f"Data/tweet_topic_{n_topics}t_{n_iterations}i"+nouns_only+".csv", index=False)


Models/lda_35t_10000i

Models/lda_50t_10000i
